In [ ]:
# import apache_beam as beam
# from random import randint


# class SplitRow(beam.DoFn):
#     def process(self, element):
#         return [element.split(",")]


# p = beam.Pipeline()

# (
#     p
#     | beam.io.ReadFromText("../../data/data.csv", skip_header_lines=1)
#     | "Split CSV" >> beam.ParDo(SplitRow())
#     | "Add visits" >> beam.Map(lambda r: ",".join([r[0], str(randint(0, 100))]))
#     | beam.io.WriteToText(file_path_prefix="../../data/data_side_input", file_name_suffix=".csv", header="Name,Visits")
# )

# Run to regenerate the side input
# p.run()


In [ ]:
import apache_beam as beam
import csv

p = beam.Pipeline()

side_input = dict()

with open("../../data/data_side_input.csv") as f:
    reader = csv.DictReader(f)
    next(reader)
    for row in reader:
        side_input[row['Name']] = row['Visits']


class FindUsersWithNoVisits(beam.DoFn):
    def process(self, element, side_input):
        row = element.split(",")
        visits = side_input.get(row[0])
        if visits != None and int(visits) < 1:
            return [element]


(
    p
    | beam.io.ReadFromText("../../data/data.csv", skip_header_lines=1)
    | "Find users with 0 visits" >> beam.ParDo(FindUsersWithNoVisits(), side_input)
    | beam.Map(print)
)

p.run()
